<a href="https://colab.research.google.com/github/Luvishka-Seeboo/WeatherWise-/blob/main/Luvishka_Seeboo_WeatherWise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install requests ipywidgets IPython display

In [ ]:
import requests
import ipywidgets as widgets
from IPython.display import display, clear_output

def get_weather(city, api_key):
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"q": f"{city},MU", "appid": api_key, "units": "metric"}
    response = requests.get(url, params=params)
    response.raise_for_status()
    d = response.json()
    return {
        "City":       d["name"],
        "Condition":  d["weather"][0]["description"].title(),
        "Temp":       f"{d['main']['temp']:.1f}°C",
        "FeelsLike":  f"{d['main']['feels_like']:.1f}°C",
        "Humidity":   f"{d['main']['humidity']}%",
        "WindSpeed":  f"{d['wind']['speed']:.1f} m/s",
        "Icon":       d["weather"][0]["icon"]
    }

def make_card(title, value, bg_gradient):
    return widgets.HTML(
        f"""
        <div class="card" style="
            background: {bg_gradient};
            border-radius: 15px;
            padding: 12px;
            text-align: center;
            box-shadow: 0 8px 16px rgba(0,0,0,0.2);
            transition: transform .2s;
        ">
          <div style="font-size:14px;color:#fff;">{title}</div>
          <div style="font-size:26px;font-weight:bold;color:#fff;margin-top:6px;">{value}</div>
        </div>
        """,
        layout=widgets.Layout(width='130px', height='100px')
    )

def on_click(b):
    clear_output(wait=True)
    display(main_ui)
    key = api_key.value.strip()
    city = city_dd.value
    if not key:
        header.value = "<h3 style='color:#e74c3c;'>❌ Enter API key!</h3>"
        grid.children = []
        return
    try:
        w = get_weather(city, key)
    except Exception as e:
        header.value = f"<h3 style='color:#e74c3c;'>❌ {e}</h3>"
        grid.children = []
        return

    icon_url = f"http://openweathermap.org/img/wn/{w['Icon']}@4x.png"
    header.value = f"""
      <div style="display:flex;align-items:center;">
        <div style="
          background: linear-gradient(135deg,#89f7fe,#66a6ff);
          border-radius:50%;
          padding:12px; margin-right:12px;
          box-shadow: 0 8px 16px rgba(0,0,0,0.15);
        ">
          <img src="{icon_url}" width="60">
        </div>
        <div>
          <h2 style="margin:0;color:#34495e;">{w['City']} 🇲🇺</h2>
          <div style="color:#7f8c8d;font-size:14px;">{w['Condition']}</div>
        </div>
      </div>
    """

    grid.children = [
        make_card("Temp",       w["Temp"],      "linear-gradient(135deg,#ff9a9e,#fad0c4)"),
        make_card("Feels Like", w["FeelsLike"], "linear-gradient(135deg,#a18cd1,#fbc2eb)"),
        make_card("Humidity",   w["Humidity"],  "linear-gradient(135deg,#89f7fe,#66a6ff)"),
        make_card("Wind",       w["WindSpeed"], "linear-gradient(135deg,#f6d365,#fda085)")
    ]

# ——— Style injection for hover effect ———
style = widgets.HTML("""
<style>
.card:hover {
  transform: translateY(-6px) scale(1.02);
  box-shadow: 0 12px 24px rgba(0,0,0,0.2);
}
</style>
""")
display(style)

# ——— Inputs ———
api_key = widgets.Password(description="API Key:", layout=widgets.Layout(width='340px'))
city_dd = widgets.Dropdown(
    options=["Port Louis","Curepipe","Beau Bassin-Rose Hill","Vacoas-Phoenix",
             "Quatre Bornes","Triolet","Rose Belle","Petit Raffray","Mahébourg","Moka"],
    description="City:", layout=widgets.Layout(width='340px')
)
btn = widgets.Button(description="Show Weather", button_style='info', layout=widgets.Layout(width='160px'))
btn.on_click(on_click)

# ——— Display areas ———
header = widgets.HTML("<h3 style='color:#2c3e50;'>Enter API key & select a city</h3>",
                      layout=widgets.Layout(padding='8px'))
grid = widgets.HBox([], layout=widgets.Layout(justify_content='space-around', padding='12px'))

# ——— Dashboard layout ———
banner = widgets.HTML("""
  <div style="
    background: linear-gradient(135deg,#667eea,#764ba2);
    color: white;
    padding: 14px 20px;
    border-radius: 12px;
    box-shadow: 0 8px 16px rgba(0,0,0,0.25);
    margin-bottom:10px;
  ">
    <h1 style="margin:0;font-weight:300;">🇲🇺 Mauritius Weather</h1>
  </div>
""")

controls = widgets.VBox([api_key, city_dd, btn], layout=widgets.Layout(padding='10px',
    border='1px solid #ddd', border_radius='12px', box_shadow='0 4px 8px rgba(0,0,0,0.1)'))

main_ui = widgets.VBox([
    banner,
    widgets.HBox([controls, widgets.VBox([header], layout=widgets.Layout(padding='10px'))]),
    grid
], layout=widgets.Layout(width='760px'))

display(main_ui)